## Getting and Visualizing the Stock Prices <a id="data">

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt
from indicators import *
from tensorflow import keras
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import StandardScaler

In [ ]:
#storing the symbols, starting and ending dates for training
ticker = "GOOG"
n_years = 10
end_date =  dt.datetime.today()
start_date = end_date - relativedelta(years=n_years)

In [ ]:
#creating a list with the input symbol and "SPY" to pull in all days the stock market was open
symbols = [ticker, "SPY"]

#reading in the stock data (using adjusted stock prices to account for stock splits)
df = yf.download(symbols, start=start_date, end=end_date, auto_adjust=True)

df.head()

In [ ]:
#only keeping Adjusted Closing Stock Price for Google
prices = df['Close'].filter(items=[ticker], axis=1)

#forward and back filling missing prices 
prices.fillna(method='ffill', inplace=True) #forward-filling missing prices
prices.fillna(method='bfill', inplace=True) #back-filling missing prices

prices.head()

In [ ]:
#plotting the adjusted closing prices
prices.plot()
plt.title("{} Adjusted Closing Prices for last {} Years".format(ticker,n_years))
plt.ylabel("Price")
plt.show()

## Creating the Features <a id="features">

In [ ]:
#indicator functions
def generate_indicators(prices, n):
    #generating price Simple Moving Average ratio
    price_sma_df = price_sma_ratio(prices, n)

    #generating the Bollinger Bands indicator
    bb_df = bollinger_bands(prices, n)

    #generating the volatility indicator
    vol_df = volatility(prices, n)
    
    #creating features dataframe for training the regressor
    indicator_names = ["Price/SMA", "Bollinger Bands", "Volatility"]
    indicators_df = price_sma_df.join(bb_df,rsuffix="1") \
                                .join(vol_df,rsuffix="2")
    indicators_df.columns = indicator_names

    #adding column for the closing price one day prior
    indicators_df["Previous Price"] = prices.shift(1).values

    #backfilling indicators
    indicators_df.fillna(method='bfill', inplace=True)
    
    return indicators_df

In [ ]:
#splitting the prices into train and test sets
ss = StandardScaler()
cutoff_idx = int(prices.shape[0] * 0.8)
y_train = pd.DataFrame(ss.fit_transform(prices[:cutoff_idx]),index=prices.index[:cutoff_idx], columns=[ticker])
y_test = pd.DataFrame(ss.fit_transform(prices[cutoff_idx:]),index=prices.index[cutoff_idx:], columns=[ticker])

#generating the indicators
n = 10
x_train = generate_indicators(y_train, n)
x_test = generate_indicators(y_test, n)

x_train.head()

## Creating the Model <a id="model">

In [ ]:
#initializing the model
model = keras.Sequential([
    keras.layers.Dense(128, input_dim=x_train.shape[1], activation='relu'), 
    keras.layers.Dropout(0.3),
    keras.layers.Dense(32,activation='relu'),
    keras.layers.Dropout(0.15),
    keras.layers.Dense(1)   
])

In [ ]:
#printing the model summary
model.summary()

In [ ]:
#compiling the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

In [ ]:
history = model.fit(x_train, y_train, epochs=100)

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
#plotting the predicted and actual normalized test prices
y_pred = model.predict(x_test)
plt.plot(y_test.index, y_pred,  label="Predicted")
plt.plot(y_test.index, y_test, label="Actual")
plt.title("Predicted vs Actual Prices")
plt.ylabel("Normalized Price")
plt.xlabel("Date")
plt.legend()
plt.show()

In [ ]:
#calculating the Mean Absolute Error of the non-normalized prices
pred_prices = ss.inverse_transform(y_pred)
act_prices = prices[cutoff_idx:]
abs_diff = np.abs(pred_prices.flatten() - act_prices.values)
mae = np.mean(abs_diff)

print("Mean Absolute Error of Stock Prices: ${:,.2f}".format(mae))